# Forth Problem

## Imports

In [81]:
import sys
sys.path.append("../../")
sys.path.append("../")
from dlpmln import DeepLPMLN
import torch

import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

from data_function import create_data_sample,format_dataList,format_observations,add_test,carry_test

## dprogram

Specify the DeepLPMLN program.

In [82]:
dprogram='''
num1(1,num1_1).
num1(0,num1_0).
num2(1,num2_1).
num2(0,num2_0).
1{carry(0,Carry): Carry=0..1}1.
nn(m1(A, B, Carry, 1), result, [0,1,2,3,4,5,6,7,8,9]) :- num1(P,A), num2(P,B), Carry=0..1.
nn(m2(A, B, Carry, 1), carry, [0,1]) :- num1(P,A), num2(P,B), Carry=0..1.

result(P,X) :- num1(P, A), num2(P, B), carry(P, Carry), result(A,B,Carry,0,X).
carry(P+1,X) :- num1(P, A), num2(P, B), carry(P, Carry), carry(A,B,Carry,0,X).
'''

## Neural Network Class

Create the neural network class that will be used for our data.

In [83]:
class FC(nn.Module):

    def __init__(self, *sizes):
        super(FC, self).__init__()
        layers = []
        for i in range(len(sizes)-2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(sizes[-2], sizes[-1]))
        layers.append(nn.Softmax(1))
        self.nn = nn.Sequential(*layers)

    def forward(self, x):
        return self.nn(x)

## Neural Network Instantiation
- Instantiate neural networks.
- Map neural networks into the NNMapping function called 'function'.
- Specify the optimizers for each network (we use the Adam optimizer here).

In [84]:
m1 = FC(30,25,10) # network for adding the numbers
m2 = FC(30,5,2)   # network for finding the carry out


functions = {'m1':m1, 'm2':m2} #NNMapping

optimizers = {'m1':torch.optim.Adam(m1.parameters(), lr=0.01),'m2':torch.optim.Adam(m2.parameters(), lr=0.01)}


Below are 20 samples of data.

In [85]:
for i in range(20):
    print(create_data_sample()[0])

add([2,9],[2,5],0,[0,5,4]).
add([0,1],[7,9],1,[0,8,1]).
add([3,1],[1,9],1,[0,5,1]).
add([8,5],[2,6],0,[1,1,1]).
add([1,9],[4,5],0,[0,6,4]).
add([5,8],[6,7],0,[1,2,5]).
add([6,2],[8,3],0,[1,4,5]).
add([7,1],[2,2],1,[0,9,4]).
add([8,9],[4,7],1,[1,3,7]).
add([9,2],[2,8],1,[1,2,1]).
add([3,3],[3,1],0,[0,6,4]).
add([6,5],[4,0],1,[1,0,6]).
add([5,7],[8,2],0,[1,3,9]).
add([3,3],[4,9],0,[0,8,2]).
add([7,0],[9,4],1,[1,6,5]).
add([4,8],[7,5],0,[1,2,3]).
add([2,3],[6,2],0,[0,8,5]).
add([4,6],[5,1],1,[0,9,8]).
add([1,6],[5,2],0,[0,6,8]).
add([3,9],[6,7],1,[1,0,7]).


In [86]:
print('\n\n')

obs,str_list=create_data_sample() #data sample the same as DeepProbLog used 
print(obs+'\n\n')

dl=format_dataList(obs,str_list) # dataList item
print(dl)

o=format_observations(obs,str_list) # obsList item
print(o)




add([6,7],[8,4],1,[1,5,2]).


{'num1_0,num2_0,0': tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_0,num2_0,1': tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_1,num2_1,0': tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_1,num2_1,1': tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])}

    :-not carry(0,1).
    :-not carry(2,1).
    :-not result(1,5).
    :-not result(0,2).
    


## Create dataList and obsList 

This the code that will use format_dataList() and format_observations() to build the data set that our DeepLPMLN object will train on.

In [87]:
dataList = []
obsList = []

train_size=500
test_size=500


for i in range(train_size):
    
    obs,str_list=create_data_sample()
    
    dataList.append(format_dataList(obs,str_list))
    obsList.append(format_observations(obs,str_list))

## Create test dataset and dataLoader for neural network "m1"

In [88]:
add_test_dataset=add_test(test_size)

add_test_dataloader=DataLoader(add_test_dataset,batch_size=4,shuffle=True)

carry_test_dataset=carry_test(test_size)

carry_test_dataloader=DataLoader(carry_test_dataset,batch_size=4,shuffle=True)



## Create DeepLPMLN object

In [89]:
dlpmlnObj = DeepLPMLN(dprogram, functions, optimizers, dynamicMVPP=False)
#dlpmlnObj.device='cpu' #put the training on the CPU

## Training and Testing 

In [90]:
print('training...')

for i in range(40):
    time1 = time.time()
    dlpmlnObj.learn(dataList=dataList, obsList=obsList, epoch=1)
    time2 = time.time()
    dlpmlnObj.testNN("m1", add_test_dataloader) #test m1 network
    dlpmlnObj.testNN("m2", carry_test_dataloader) #test m2 network
    print("--- train time: %s seconds ---" % (time2 - time1))
    print("--- test time: %s seconds ---" % (time.time() - time2))

training...
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 21%
Test Accuracy on NN Only for m2: 93%
--- train time: 59.55068755149841 seconds ---
--- test time: 0.1405956745147705 seconds ---
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 42%
Test Accuracy on NN Only for m2: 94%
--- train time: 59.649752616882324 seconds ---
--- test time: 0.1411747932434082 seconds ---
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 64%
Test Accuracy on NN Only for m2: 95%
--- train time: 59.609055042266846 seconds ---
--- test time: 0.14249491691589355 seconds ---
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 70%
Test Accuracy on NN Only for m2: 94%
--- train time: 59.77775168418884 seconds ---
--- test time: 0.14168334007263184 seconds ---
Training for epoch 1 ...
Test Accuracy on NN Only for m1: 82%
Test Accuracy on NN Only for m2: 96%
--- train time: 60.18323612213135 seconds ---
--- test time: 0.1432783603668213 seconds ---
Training for epoch 1 ...
Test

KeyboardInterrupt: 